In [ ]:
from fastai.collab import *
from fastai.tabular.all import *


path = untar_data(URLs.ML_100k)
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])

In [ ]:
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
last_skywalker = np.array([0.98,0.9,-0.9])
user1 = np.array([0.9,0.8,-0.6])

(user1*last_skywalker).sum()

2.1420000000000003

In [ ]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,758,Leaving Las Vegas (1995),2
1,837,Leaving Las Vegas (1995),1
2,65,Independence Day (ID4) (1996),4
3,762,"MatchMaker, The (1997)",1
4,126,I Know What You Did Last Summer (1997),1
5,38,Heathers (1989),5
6,263,Some Like It Hot (1959),4
7,305,Clerks (1994),4
8,403,Beavis and Butt-head Do America (1996),1
9,532,Mallrats (1995),4


In [ ]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [ ]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([ 0.4351,  2.3266, -0.8269, -0.4757, -0.7687])

In [ ]:
one_hot_3

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
user_factors[3]

tensor([ 0.4351,  2.3266, -0.8269, -0.4757, -0.7687])

In [ ]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [ ]:
class DotProduct(Module):
 def __init__(self, n_users, n_movies, n_factors):
  self.user_factors = Embedding(n_users, n_factors)
  self.movie_factors = Embedding(n_movies, n_factors)

 def forward(self, x):
  users = self.user_factors(x[:,0])
  movies = self.movie_factors(x[:,1])
  return (users * movies).sum(dim=1)

In [ ]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [ ]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.315904,1.332944,00:10
1,1.008039,1.098195,00:10
2,0.889742,0.978030,00:10
3,0.767489,0.912768,00:10
4,0.739915,0.884933,00:10


In [ ]:
class DotProduct(Module):
 def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
  self.user_factors = Embedding(n_users, n_factors)
  self.movie_factors = Embedding(n_movies, n_factors)
  self.y_range = y_range

 def forward(self, x):
  users = self.user_factors(x[:,0])
  movies = self.movie_factors(x[:,1])
  return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.966152,1.004723,00:10
1,0.654193,0.938037,00:10
2,0.466896,0.938722,00:17
3,0.350753,0.937291,00:17
4,0.358570,0.935842,00:18


In [ ]:
class DotProductBias(Module):
 def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
  self.user_factors = Embedding(n_users, n_factors)
  self.user_bias = Embedding(n_users, 1)
  self.movie_factors = Embedding(n_movies, n_factors)
  self.movie_bias = Embedding(n_movies, 1)
  self.y_range = y_range

 def forward(self, x):
  users = self.user_factors(x[:,0])
  movies = self.movie_factors(x[:,1])
  res = (users * movies).sum(dim=1, keepdim=True)
  res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
  return sigmoid_range(res, *self.y_range)

In [ ]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.880544,0.936517,00:11
1,0.602460,0.898228,00:11
2,0.396920,0.940116,00:11
3,0.319613,0.949451,00:18
4,0.303496,0.952065,00:16


In [ ]:
#### pass the wd for the l2 generalization of the weights

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.934652,0.976401,00:11
1,0.776436,0.916477,00:11
2,0.593627,0.880384,00:11
3,0.509149,0.852168,00:10
4,0.502798,0.843660,00:11


In [ ]:
class T(Module):
 def __init__(self): self.a = nn.Parameter(torch.ones(3))

L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [ ]:
class T(Module):
 def __init__(self): self.a = nn.Linear(1, 3, bias=False)
t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.6744],
        [ 0.2105],
        [ 0.8299]], requires_grad=True)]

In [ ]:
def create_params(size):
 return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [ ]:
class DotProductBias(Module):
 def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
  self.user_factors = create_params([n_users, n_factors])
  self.user_bias = create_params([n_users])
  self.movie_factors = create_params([n_movies, n_factors])
  self.movie_bias = create_params([n_movies])
  self.y_range = y_range

 def forward(self, x):
  users = self.user_factors[x[:,0]]
  movies = self.movie_factors[x[:,1]]
  res = (users*movies).sum(dim=1)
  res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
  return sigmoid_range(res, *self.y_range)

In [ ]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.873572,0.929105,00:12
1,0.653024,0.888332,00:12
2,0.546775,0.865015,00:11
3,0.465600,0.844923,00:11
4,0.441545,0.842129,00:11


In [ ]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idxs]

['Children of the Corn: The Gathering (1996)',
 'Island of Dr. Moreau, The (1996)',
 'Crow: City of Angels, The (1996)',
 'Bio-Dome (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)']

In [ ]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.913385,0.942227,00:11
1,0.656131,0.880112,00:11
2,0.518322,0.859260,00:11
3,0.466844,0.839888,00:11
4,0.452208,0.834836,00:11


In [ ]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [ ]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Star Wars (1977)',
 'Good Will Hunting (1997)',
 'Titanic (1997)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)"]

In [ ]:
### find the closest movie to the one movie
### as the distance between the embeddings should be the same

movie_factoes = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Silence of the Lambs, The (1991)']
distances = nn.CosineSimilarity(dim=1)(movie_factoes, movie_factoes[idx][None])
print(distances)
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

tensor([-0.0275, -0.1507,  0.0552,  ..., -0.2419,  0.1573,  0.1177],
       grad_fn=<SumBackward1>)


'For Whom the Bell Tolls (1943)'

In [ ]:
embs = get_emb_sz(dls)
embs

class CollabNN(Module):
 def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
  self.user_factors = Embedding(*user_sz)
  self.item_factors = Embedding(*item_sz)
  self.layers = nn.Sequential(
  nn.Linear(user_sz[1]+item_sz[1], n_act),
  nn.ReLU(),
  nn.Linear(n_act, 1))
  self.y_range = y_range

 def forward(self, x):
  embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
  x = self.layers(torch.cat(embs, dim=1))
  return sigmoid_range(x, *self.y_range)

In [ ]:
model = CollabNN(*embs)

In [ ]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.927449,0.955998,00:15
1,0.850204,0.908463,00:14
2,0.809149,0.884506,00:15
3,0.775829,0.870661,00:14
4,0.767706,0.868002,00:14


In [ ]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.970635,0.977105,00:16
1,0.893720,0.913893,00:15
2,0.828612,0.880880,00:15
3,0.761903,0.863174,00:15
4,0.728088,0.858221,00:15


In [ ]:
@delegates(TabularModel)
class EmbeddingNN(TabularModel):
 def __init__(self, emb_szs, layers, **kwargs):
 super().__init__(emb_szs, layers=layers, n_cont=0, out_sz=1, **kwargs)

IndentationError: expected an indented block after function definition on line 3 (<ipython-input-37-56d19a23f4b0>, line 4)